## Load and clean the data

In [1]:
import pandas as pd

### COVID data

The COVID-19 pandemic has affected everyone in the United States, and the government's response the the pandemic took center stage in the election. Therefore, I want to use data on COVID cases and deaths in my analysis to determine if there was a trend between counties that were hit particularly hard by COVID and their voting preferences. 

This data will consist of cumulative case and death totals from the COVID-19 pandemic. This data also includes recent population totals that I will use to calculate COVID related metrics.

In [3]:
covid_cases = pd.read_csv('Data/covid_confirmed_usafacts.csv', index_col='countyFIPS')
covid_deaths = pd.read_csv('Data/covid_deaths_usafacts.csv', index_col='countyFIPS')
county_populations = pd.read_csv('Data/covid_county_population_usafacts.csv', index_col='countyFIPS')

In [4]:
covid_cases.head()

,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20
countyFIPS,,,,,,,,,,,,,,,,,,,,,
0,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001,Autauga County,AL,1,0,0,0,0,0,0,0,...,2159,2173,2186,2197,2212,2230,2242,2267,2283,2304
1003,Baldwin County,AL,1,0,0,0,0,0,0,0,...,6940,6966,6985,6995,7061,7097,7134,7188,7226,7263
1005,Barbour County,AL,1,0,0,0,0,0,0,0,...,1060,1061,1065,1074,1079,1080,1090,1092,1095,1098
1007,Bibb County,AL,1,0,0,0,0,0,0,0,...,873,878,883,890,897,907,917,924,926,932


#### Remove the "Statewide Unallocated" rows. These have a countyFIPS label of 0.

In [5]:
covid_cases.drop(labels=0, inplace=True)
covid_deaths.drop(labels=0, inplace=True)
county_populations.drop(labels=0, inplace=True)

In [6]:
print(covid_cases.shape)
print(covid_deaths.shape)
print(county_populations.shape)

(3145, 296)
(3145, 296)
(3144, 3)


For some reason the county_populations dataframe has one less row than the other two dataframes. I'm going to join the county_populations dataframe with one of the others to see which row is present in the others but missing in county_populations.

In [7]:
test_df = covid_cases.join(county_populations, lsuffix='_cases', rsuffix='_pop')

In [8]:
test_df[test_df.isna()['population']]

,County Name_cases,State_cases,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,County Name_pop,State_pop,population
countyFIPS,,,,,,,,,,,,,,,,,,,,,
1,New York City Unallocated/Probable,NY,36,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


It turns out that there is a row in the covid_cases and covid_deaths dataframes for a county titled "New York City Unallocated/Probable" that is not present in the county_populations dataframe. I will remove this row and now each of the three dataframes will have the same number of rows.

In [12]:
covid_cases.drop(labels=1, inplace=True)
covid_deaths.drop(labels=1, inplace=True)

In [13]:
print(covid_cases.shape)
print(covid_deaths.shape)
print(county_populations.shape)

(3144, 296)
(3144, 296)
(3144, 3)


#### Next, remove all columns other than November 1st (2 days before election)

In [16]:
covid_cases = covid_cases[['County Name', 'State', '11/1/20']]

In [17]:
# Rename the column to note that it represents COVID cases
covid_cases.rename(columns={'11/1/20': 'COVID Cases'}, inplace=True)

In [18]:
covid_cases.head()

,County Name,State,COVID Cases
countyFIPS,,,
1001,Autauga County,AL,2173
1003,Baldwin County,AL,6966
1005,Barbour County,AL,1061
1007,Bibb County,AL,878
1009,Blount County,AL,2095


Do the same for COVID deaths.

In [20]:
covid_deaths = covid_deaths[['County Name', 'State', '11/1/20']]

In [21]:
# Rename the column to note that it represents deaths
covid_deaths.rename(columns={'11/1/20': 'COVID Deaths'}, inplace=True)

In [22]:
covid_deaths.head()

,County Name,State,COVID Deaths
countyFIPS,,,
1001,Autauga County,AL,30
1003,Baldwin County,AL,71
1005,Barbour County,AL,9
1007,Bibb County,AL,15
1009,Blount County,AL,25


#### Next, join these three dataframes into a single dataframe.

In [23]:
covid_data = covid_cases.join(covid_deaths['COVID Deaths'])

In [25]:
# Capitalize population column name
county_populations.rename(columns={'population': 'Population'}, inplace=True)
covid_data = covid_data.join(county_populations['Population'])

In [26]:
print(covid_data.shape)
covid_data.head()

(3144, 5)


,County Name,State,COVID Cases,COVID Deaths,Population
countyFIPS,,,,,
1001,Autauga County,AL,2173,30,55869
1003,Baldwin County,AL,6966,71,223234
1005,Barbour County,AL,1061,9,24686
1007,Bibb County,AL,878,15,22394
1009,Blount County,AL,2095,25,57826


#### Create two new columns for cases and deaths per 100,000

In [27]:
def metric_per_100k(value, population):
    '''
    Takes a value and a population and computes
    the value per 100,000 residents.
    '''
    if population == 0:
        return 0
    else:
        return round((value / population) * 100000, 2)

In [28]:
covid_data['COVID Cases per 100k'] = covid_data.apply(lambda x: metric_per_100k(x['COVID Cases'], 
                                                                                x['Population']), axis=1)
covid_data['COVID Deaths per 100k'] = covid_data.apply(lambda x: metric_per_100k(x['COVID Deaths'], 
                                                                                x['Population']), axis=1)

In [29]:
covid_data.head()

,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k
countyFIPS,,,,,,,
1001,Autauga County,AL,2173,30,55869,3889.46,53.70
1003,Baldwin County,AL,6966,71,223234,3120.49,31.81
1005,Barbour County,AL,1061,9,24686,4297.98,36.46
1007,Bibb County,AL,878,15,22394,3920.69,66.98
1009,Blount County,AL,2095,25,57826,3622.94,43.23


#### In order to properly join with the other datasets, countyFIPS will need to be padded with zeros to match the length of the FIPS county codes in the other datasets.

In [31]:
def pad_fips(code):
    """
    Function prepends zeros to the beginning of countyFIPS 
    codes whose lengths are less than five characters long.
    Returns the code as a string.
    """
    code_str = str(code)
    while len(code_str) < 5:
        code_str = '0' + code_str
    return code_str

In [32]:
covid_data.index = covid_data.index.map(pad_fips)

In [33]:
covid_data.head()

,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k
countyFIPS,,,,,,,
01001,Autauga County,AL,2173,30,55869,3889.46,53.70
01003,Baldwin County,AL,6966,71,223234,3120.49,31.81
01005,Barbour County,AL,1061,9,24686,4297.98,36.46
01007,Bibb County,AL,878,15,22394,3920.69,66.98
01009,Blount County,AL,2095,25,57826,3622.94,43.23


In [34]:
# Save this cleaned dataset
covid_data.to_csv('Data/cleaned_covid_data.csv')

### Unemployment data

This data will be the unemployment rate in every county. The most recent numbers I could find were for September 2020.

Jobs have historically been a major factor in determining who will win the presidency, particularly when an incumbent is running for re-election. This data tries the capture the general health of the jobs market as close to the election as possible.

In [41]:
county_unemployment = pd.read_excel('Data/unemployment_data.xlsx', header=4)

In [42]:
county_unemployment.head()

,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CN0100100000000,1.0,1.0,"Autauga County, AL",Aug-19,26079,25368,711,2.7
2,CN0100300000000,1.0,3.0,"Baldwin County, AL",Aug-19,97939,95367,2572,2.6
3,CN0100500000000,1.0,5.0,"Barbour County, AL",Aug-19,8652,8322,330,3.8
4,CN0100700000000,1.0,7.0,"Bibb County, AL",Aug-19,8670,8403,267,3.1


In [43]:
county_unemployment.drop(labels=0, inplace=True) # drop the first row because it is null

In [44]:
county_unemployment.head()

,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
1,CN0100100000000,1.0,1.0,"Autauga County, AL",Aug-19,26079,25368,711,2.7
2,CN0100300000000,1.0,3.0,"Baldwin County, AL",Aug-19,97939,95367,2572,2.6
3,CN0100500000000,1.0,5.0,"Barbour County, AL",Aug-19,8652,8322,330,3.8
4,CN0100700000000,1.0,7.0,"Bibb County, AL",Aug-19,8670,8403,267,3.1
5,CN0100900000000,1.0,9.0,"Blount County, AL",Aug-19,25309,24641,668,2.6


In [45]:
county_unemployment.shape

(45069, 9)

#### Take the rows for September 2020

In [46]:
county_unemployment = county_unemployment[county_unemployment['Period'] == 'Sep-20 p']

In [47]:
county_unemployment.head()

,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
41848,CN0100100000000,1.0,1.0,"Autauga County, AL",Sep-20 p,25241,23925,1316,5.2
41849,CN0100300000000,1.0,3.0,"Baldwin County, AL",Sep-20 p,96940,91585,5355,5.5
41850,CN0100500000000,1.0,5.0,"Barbour County, AL",Sep-20 p,9528,8708,820,8.6
41851,CN0100700000000,1.0,7.0,"Bibb County, AL",Sep-20 p,8587,8021,566,6.6
41852,CN0100900000000,1.0,9.0,"Blount County, AL",Sep-20 p,24525,23537,988,4


#### Split the "County Name/State Abbreviation" column into separate columns

In [48]:
county_unemployment['County Name'] = county_unemployment['County Name/State Abbreviation']\
                                    .apply(lambda name: name.split(',')[0].strip())

county_unemployment['State'] = county_unemployment['County Name/State Abbreviation']\
                                    .apply(lambda name: name.split(',')[-1].strip())

In [50]:
# Drop the irrelevant columns
county_unemployment.drop(labels=['Code', 'Code.1', 'Force', 'Employed', 'Unemployed', 'Period',
                                 'County Name/State Abbreviation'], axis=1, inplace=True)

In [52]:
# Rename the (%) column as Unemployment Rate
county_unemployment.rename(columns={'(%)': 'Unemployment Rate (%) - Sept'}, inplace=True)

In [53]:
county_unemployment.head()

,LAUS Code,Unemployment Rate (%) - Sept,County Name,State
41848,CN0100100000000,5.2,Autauga County,AL
41849,CN0100300000000,5.5,Baldwin County,AL
41850,CN0100500000000,8.6,Barbour County,AL
41851,CN0100700000000,6.6,Bibb County,AL
41852,CN0100900000000,4,Blount County,AL


In [54]:
county_unemployment.shape

(3219, 4)

#### Remove Puerto Rico and reformat District of Columbia to DC for consistency with the other dataset

In [55]:
county_unemployment = county_unemployment[county_unemployment['State'] != 'PR']

In [57]:
# Reformat District of Columbia
county_unemployment['State'] = county_unemployment['State'].apply(lambda state: 'DC' if state == 'District of Columbia' else state)

In [58]:
county_unemployment['State'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [59]:
county_unemployment

,LAUS Code,Unemployment Rate (%) - Sept,County Name,State
41848,CN0100100000000,5.2,Autauga County,AL
41849,CN0100300000000,5.5,Baldwin County,AL
41850,CN0100500000000,8.6,Barbour County,AL
41851,CN0100700000000,6.6,Bibb County,AL
41852,CN0100900000000,4,Blount County,AL
...,...,...,...,...
44984,CN5603700000000,7,Sweetwater County,WY
44985,CN5603900000000,4.2,Teton County,WY
44986,CN5604100000000,6.4,Uinta County,WY
44987,CN5604300000000,4.7,Washakie County,WY


#### Next slice the content of the LAUS Code column to match the 5 number FIPS code and set this column to this dataframe's index.

In [60]:
# Rename LAUS Code to countyFIPS for consistency
county_unemployment.rename(columns={'LAUS Code': 'countyFIPS'}, inplace=True)

In [61]:
county_unemployment['countyFIPS'] = county_unemployment['countyFIPS'].apply(lambda code: code[2:7]) # slice the column

In [62]:
county_unemployment.set_index('countyFIPS', inplace=True)

In [63]:
county_unemployment

,Unemployment Rate (%) - Sept,County Name,State
countyFIPS,,,
01001,5.2,Autauga County,AL
01003,5.5,Baldwin County,AL
01005,8.6,Barbour County,AL
01007,6.6,Bibb County,AL
01009,4,Blount County,AL
...,...,...,...
56037,7,Sweetwater County,WY
56039,4.2,Teton County,WY
56041,6.4,Uinta County,WY


In [64]:
# Save this cleaned unemployment data
county_unemployment.to_csv('Data/cleaned_unemployment_data.csv')

#### Now join the covid and unemployment dataframes to figure out why the unemployment data has 3 fewer rows than the covid dataframe.

In [65]:
test_df = covid_data.join(county_unemployment, lsuffix='_covid', rsuffix='_unemp')

In [66]:
test_df[test_df.isna()['State_unemp']]

,County Name_covid,State_covid,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,County Name_unemp,State_unemp
countyFIPS,,,,,,,,,,
02270,Wade Hampton Census Area,AK,0,0,0,0.0,0.0,NaN,NaN,NaN
06000,Grand Princess Cruise Ship,CA,21,0,0,0.0,0.0,NaN,NaN,NaN
15005,Kalawao County,HI,0,0,86,0.0,0.0,NaN,NaN,NaN


It turns out that the covid dataset contains rows for small areas of interest that are not included in the Bureau of Labor Statistics data. I will drop these examples because they will not appear in my 2020 election data.

### From here on, I will maintain a master dataframe that will be built incrementally as I add more data sources.

In [67]:
# Join the covid_data with the county_unemployment data
master_data = covid_data.join(county_unemployment['Unemployment Rate (%) - Sept'])

In [68]:
print(master_data.shape)

(3144, 8)


In [69]:
# Drop the rows that do not appear in the newly joined unemployment column
master_data.dropna(inplace=True)

In [70]:
print(master_data.shape)
master_data.head()

(3141, 8)


,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept
countyFIPS,,,,,,,,
01001,Autauga County,AL,2173,30,55869,3889.46,53.70,5.2
01003,Baldwin County,AL,6966,71,223234,3120.49,31.81,5.5
01005,Barbour County,AL,1061,9,24686,4297.98,36.46,8.6
01007,Bibb County,AL,878,15,22394,3920.69,66.98,6.6
01009,Blount County,AL,2095,25,57826,3622.94,43.23,4


### Median Income and Poverty Rate data

These are 2018 US Census estimates of the Median Household Income and Poverty Rates across counties. Although this data is not as recent as 2020, I'm making the assumption that median income and poverty rates have not drastically changed since 2018. In normal times it would probably be safe to assume that median income and poverty rates do not change much over a two year period. However, with the COVID-19 pandemic it is likely that these numbers have changed. Nevertheless, these metrics are meant to capture the long term state of a county's economy. The richest and poorest counties will not suddenly switch places due to the pandemic. I have attempted to capture the short term effects of the pandemic in the unemployment data that I have already included.

In [71]:
income_poverty_data = pd.read_excel('Data/median_income_poverty.xls', header=3)

In [72]:
income_poverty_data.head()

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound.1,90% CI Upper Bound.1,...,90% CI Upper Bound.5,Median Household Income,90% CI Lower Bound.6,90% CI Upper Bound.6,"Poverty Estimate, Age 0-4",90% CI Lower Bound.7,90% CI Upper Bound.7,"Poverty Percent, Age 0-4",90% CI Lower Bound.8,90% CI Upper Bound.8
0,0,0,US,United States,41852315,41619366,42085264,13.1,13,13.2,...,17.2,61937,61843,62031,3758704,3714862,3802546,19.5,19.3,19.7
1,1,0,AL,Alabama,801758,785668,817848,16.8,16.5,17.1,...,23.7,49881,49123,50639,73915,69990,77840,26,24.6,27.4
2,1,1,AL,Autauga County,7587,6334,8840,13.8,11.5,16.1,...,23.9,59338,53628,65048,.,.,.,.,.,.
3,1,3,AL,Baldwin County,21069,17390,24748,9.8,8.1,11.5,...,16.9,57588,54437,60739,.,.,.,.,.,.
4,1,5,AL,Barbour County,6788,5662,7914,30.9,25.8,36,...,45.9,34382,31157,37607,.,.,.,.,.,.


#### Select the relevant columns

In [75]:
income_poverty_data.columns

Index(['State FIPS Code', 'County FIPS Code', 'Postal Code', 'Name',
       'Poverty Estimate, All Ages', '90% CI Lower Bound',
       '90% CI Upper Bound', 'Poverty Percent, All Ages',
       '90% CI Lower Bound.1', '90% CI Upper Bound.1',
       'Poverty Estimate, Age 0-17', '90% CI Lower Bound.2',
       '90% CI Upper Bound.2', 'Poverty Percent, Age 0-17',
       '90% CI Lower Bound.3', '90% CI Upper Bound.3',
       'Poverty Estimate, Age 5-17 in Families', '90% CI Lower Bound.4',
       '90% CI Upper Bound.4', 'Poverty Percent, Age 5-17 in Families',
       '90% CI Lower Bound.5', '90% CI Upper Bound.5',
       'Median Household Income', '90% CI Lower Bound.6',
       '90% CI Upper Bound.6', 'Poverty Estimate, Age 0-4',
       '90% CI Lower Bound.7', '90% CI Upper Bound.7',
       'Poverty Percent, Age 0-4', '90% CI Lower Bound.8',
       '90% CI Upper Bound.8'],
      dtype='object')

In [76]:
income_poverty_data = income_poverty_data[['State FIPS Code', 'County FIPS Code', 'Postal Code', 
                                           'Name', 'Poverty Percent, All Ages', 'Median Household Income']]

In [77]:
income_poverty_data

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Percent, All Ages",Median Household Income
0,0,0,US,United States,13.1,61937
1,1,0,AL,Alabama,16.8,49881
2,1,1,AL,Autauga County,13.8,59338
3,1,3,AL,Baldwin County,9.8,57588
4,1,5,AL,Barbour County,30.9,34382
...,...,...,...,...,...,...
3189,56,37,WY,Sweetwater County,8.4,73315
3190,56,39,WY,Teton County,6.3,99087
3191,56,41,WY,Uinta County,10,63401
3192,56,43,WY,Washakie County,11.9,55190


In [78]:
# Change column names
income_poverty_data.rename(columns={'Name': 'County Name', 'Poverty Percent, All Ages': 'Poverty Rate (%)',
                                    'Postal Code': 'State', 'Median Household Income': 
                                    'Median Household Income ($)'}, inplace=True)

In [79]:
income_poverty_data.head()

,State FIPS Code,County FIPS Code,State,County Name,Poverty Rate (%),Median Household Income ($)
0,0,0,US,United States,13.1,61937
1,1,0,AL,Alabama,16.8,49881
2,1,1,AL,Autauga County,13.8,59338
3,1,3,AL,Baldwin County,9.8,57588
4,1,5,AL,Barbour County,30.9,34382


#### Remove state level data

In [82]:
# Remove the states and the DC duplicate; States and DC have a FIPS code of 0
income_poverty_data = income_poverty_data[income_poverty_data['County FIPS Code'] != 0]

In [83]:
print(income_poverty_data.shape)
income_poverty_data.head()

(3142, 6)


,State FIPS Code,County FIPS Code,State,County Name,Poverty Rate (%),Median Household Income ($)
2,1,1,AL,Autauga County,13.8,59338
3,1,3,AL,Baldwin County,9.8,57588
4,1,5,AL,Barbour County,30.9,34382
5,1,7,AL,Bibb County,21.8,46064
6,1,9,AL,Blount County,13.2,50412


#### Combine the State and County FIPS Codes to create the unique County FIPS codes as they appear in the previous datasets

In [84]:
def combine_fips(state, county):
    """
    Function takes a state FIPS code and a county FIPS code 
    and combines them into a single unique countyFIPS code string.
    """
    state, county = str(state), str(county) # cast the codes to strings
    if len(state) == 1:
        state = '0' + state
    while len(county) < 3:
        county = '0' + county
    return state + county

In [86]:
income_poverty_data['countyFIPS'] = income_poverty_data.apply(lambda x: combine_fips(x['State FIPS Code'], 
                                                                                     x['County FIPS Code']), axis=1)

In [88]:
# Remove original FIPS columns
income_poverty_data.drop(labels=['State FIPS Code', 'County FIPS Code'], axis=1, inplace=True)

In [90]:
# Set the countyFIPS code as the dataframe's index
income_poverty_data.set_index('countyFIPS', inplace=True)

In [91]:
print(income_poverty_data.shape)
income_poverty_data.head()

(3142, 4)


,State,County Name,Poverty Rate (%),Median Household Income ($)
countyFIPS,,,,
01001,AL,Autauga County,13.8,59338
01003,AL,Baldwin County,9.8,57588
01005,AL,Barbour County,30.9,34382
01007,AL,Bibb County,21.8,46064
01009,AL,Blount County,13.2,50412


The shape of this dataframe is 3142 which means there is one row in this dataframe that is not in my master_data. Last time I had this issue because one of my datasets included Kalawao County, HI which I am not going to include in my election analysis. Before doing anything fancy, I will first check the Hawaiian counties to see if Kalawao is included.

In [92]:
income_poverty_data[income_poverty_data['State'] == 'HI']

,State,County Name,Poverty Rate (%),Median Household Income ($)
countyFIPS,,,,
15001,HI,Hawaii County,15.6,56383
15003,HI,Honolulu County,7.7,83695
15005,HI,Kalawao County,.,.
15007,HI,Kauai County,8.5,73981
15009,HI,Maui County,8.6,78503


Indeed, Kalawao County is the extra county and it doesn't even have income or poverty data. I will drop this county.

NOTE: I realize that I don't have to drop this extra value manually because it would be dropped automatically when I perform the left join with master_data. However, I want to create new clean datasets for each of master_data's smaller constituent datasets. Therefore, I will drop this extra row before saving income_poverty_data as a csv.

In [94]:
income_poverty_data.drop(labels='15005', inplace=True)

In [95]:
income_poverty_data.shape

(3141, 4)

In [97]:
# Save this cleaned median income and poverty rate dataset
income_poverty_data.to_csv('Data/cleaned_median_income_poverty.csv')

### Add the income and poverty data to the master dataframe

In [98]:
master_data = master_data.join(income_poverty_data[['Poverty Rate (%)', 'Median Household Income ($)']])

In [99]:
master_data

,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,Poverty Rate (%),Median Household Income ($)
countyFIPS,,,,,,,,,,
01001,Autauga County,AL,2173,30,55869,3889.46,53.70,5.2,13.8,59338
01003,Baldwin County,AL,6966,71,223234,3120.49,31.81,5.5,9.8,57588
01005,Barbour County,AL,1061,9,24686,4297.98,36.46,8.6,30.9,34382
01007,Bibb County,AL,878,15,22394,3920.69,66.98,6.6,21.8,46064
01009,Blount County,AL,2095,25,57826,3622.94,43.23,4,13.2,50412
...,...,...,...,...,...,...,...,...,...,...
56037,Sweetwater County,WY,585,2,42343,1381.57,4.72,7,8.4,73315
56039,Teton County,WY,814,1,23464,3469.14,4.26,4.2,6.3,99087
56041,Uinta County,WY,488,3,20226,2412.74,14.83,6.4,10,63401


### Education data

This data will be the percentage of adults with a bachelor's degree or higher measured from 2014-2018. This is the most recent data I could find, but I believe it's safe to assume that the education level of most counties has not changed drastically since a few years ago when this data was collected.

In [100]:
education_data = pd.read_excel('Data/Education.xls', header=4)

In [101]:
education_data.head()

,FIPS Code,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970",...,"Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2014-18","High school diploma only, 2014-18","Some college or associate's degree, 2014-18","Bachelor's degree or higher, 2014-18","Percent of adults with less than a high school diploma, 2014-18","Percent of adults with a high school diploma only, 2014-18","Percent of adults completing some college or associate's degree, 2014-18","Percent of adults with a bachelor's degree or higher, 2014-18"
0,0,US,United States,NaN,NaN,NaN,NaN,52373312.0,34158051.0,11650730.0,...,27.4,24.4,26948057.0,59265308.0,63365655.0,68867051.0,12.3,27.1,29.0,31.5
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,1062306.0,468269.0,136287.0,...,25.9,19.0,470043.0,1020172.0,987148.0,822595.0,14.2,30.9,29.9,24.9
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,6611.0,3757.0,933.0,...,26.9,18.0,4204.0,12119.0,10552.0,10291.0,11.3,32.6,28.4,27.7
3,1003,AL,Baldwin County,4.0,5.0,3.0,2.0,18726.0,8426.0,2334.0,...,29.3,23.1,14310.0,40579.0,46025.0,46075.0,9.7,27.6,31.3,31.3
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,8120.0,2242.0,581.0,...,21.3,10.9,4901.0,6486.0,4566.0,2220.0,27.0,35.7,25.1,12.2


#### Extract the relevant columns

In [102]:
education_data = education_data[['FIPS Code', 'State', 'Area name', 
                                 "Percent of adults with a bachelor's degree or higher, 2014-18"]]

In [103]:
education_data.rename(columns={'FIPS Code': 'countyFIPS', 
                               'Area name': 'County Name',
                              "Percent of adults with a bachelor's degree or higher, 2014-18": 
                               '% of adults with a college degree or higher'}, inplace=True)

In [104]:
education_data.head()

,countyFIPS,State,County Name,% of adults with a college degree or higher
0,0,US,United States,31.5
1,1000,AL,Alabama,24.9
2,1001,AL,Autauga County,27.7
3,1003,AL,Baldwin County,31.3
4,1005,AL,Barbour County,12.2


#### Remove non-county rows and Puerto Rico

In [105]:
# Drop the entire US level data
education_data.drop(labels=0, inplace=True)

In [106]:
education_data.shape

(3282, 4)

In [107]:
# Drop Puerto Rico
education_data = education_data[education_data['State'] != 'PR']
education_data.shape

(3203, 4)

Drop the state level data

In [108]:
# First pad the countyFIPS column
education_data['countyFIPS'] = education_data['countyFIPS'].apply(pad_fips)

In [109]:
# It appears that the state level rows have countyFIPS codes that end in 000
education_data[education_data['countyFIPS'].apply(lambda code: code[2:] == '000')]

,countyFIPS,State,County Name,% of adults with a college degree or higher
1,01000,AL,Alabama,24.9
69,02000,AK,Alaska,29.2
106,04000,AZ,Arizona,28.9
122,05000,AR,Arkansas,22.6
198,06000,CA,California,33.3
257,08000,CO,Colorado,40.1
322,09000,CT,Connecticut,38.9
331,10000,DE,Delaware,31.4
335,11000,DC,District of Columbia,57.6
337,12000,FL,Florida,29.2


In [110]:
# To remove these columns, I can simply take the rows whose countyFIPS codes do not end in 000
education_data = education_data[education_data['countyFIPS'].apply(lambda code: code[2:] != '000')]

In [111]:
education_data

,countyFIPS,State,County Name,% of adults with a college degree or higher
2,01001,AL,Autauga County,27.7
3,01003,AL,Baldwin County,31.3
4,01005,AL,Barbour County,12.2
5,01007,AL,Bibb County,11.5
6,01009,AL,Blount County,12.6
...,...,...,...,...
3199,56037,WY,Sweetwater County,22.0
3200,56039,WY,Teton County,57.4
3201,56041,WY,Uinta County,15.4
3202,56043,WY,Washakie County,21.5


#### Set the dataframe's index to the FIPS codes

In [112]:
education_data.set_index('countyFIPS', inplace=True)

In [113]:
education_data.head()

,State,County Name,% of adults with a college degree or higher
countyFIPS,,,
01001,AL,Autauga County,27.7
01003,AL,Baldwin County,31.3
01005,AL,Barbour County,12.2
01007,AL,Bibb County,11.5
01009,AL,Blount County,12.6


#### For some reason, the education_data has more rows than the master dataframe I've accumulated so far. I will join these dataframes as a test and keep the rows from education_data to determine the difference.

In [115]:
print(master_data.shape)
print(education_data.shape)

(3141, 10)
(3152, 3)


In [116]:
test_df = education_data.join(master_data, lsuffix='_edu', rsuffix='_master')

In [117]:
test_df.head()

,State_edu,County Name_edu,% of adults with a college degree or higher,County Name_master,State_master,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,Poverty Rate (%),Median Household Income ($)
countyFIPS,,,,,,,,,,,,,
01001,AL,Autauga County,27.7,Autauga County,AL,2173.0,30.0,55869.0,3889.46,53.70,5.2,13.8,59338
01003,AL,Baldwin County,31.3,Baldwin County,AL,6966.0,71.0,223234.0,3120.49,31.81,5.5,9.8,57588
01005,AL,Barbour County,12.2,Barbour County,AL,1061.0,9.0,24686.0,4297.98,36.46,8.6,30.9,34382
01007,AL,Bibb County,11.5,Bibb County,AL,878.0,15.0,22394.0,3920.69,66.98,6.6,21.8,46064
01009,AL,Blount County,12.6,Blount County,AL,2095.0,25.0,57826.0,3622.94,43.23,4,13.2,50412


In [118]:
test_df[test_df.isna()['COVID Cases']]

,State_edu,County Name_edu,% of adults with a college degree or higher,County Name_master,State_master,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,Poverty Rate (%),Median Household Income ($)
countyFIPS,,,,,,,,,,,,,
02010,AK,Aleutian Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02160,AK,Kuskokwim Division,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02201,AK,Prince of Wales-Outer Ketchikan Census Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02231,AK,Skagway-Yakutat-Angoon Census Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02232,AK,Skagway-Hoonah-Angoon Census Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02250,AK,Upper Yukon Division,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02280,AK,Wrangell-Petersburg Census Area,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15005,HI,Kalawao County,24.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30113,MT,Yellowstone National Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
test_df[test_df.isna()['COVID Cases']].index

Index(['02010', '02160', '02201', '02231', '02232', '02250', '02280', '15005',
       '30113', '51515', '51560'],
      dtype='object', name='countyFIPS')

It appears that education_data contains a number of rows that do not appear in master_data. I will drop these rows.

In [120]:
education_data = education_data.dropna().drop(labels='15005')
education_data.shape

(3141, 3)

In [121]:
# Save education_data
education_data.to_csv('Data/cleaned_education_data.csv')

### Join master_data and education_data.

In [122]:
master_data = master_data.join(education_data["% of adults with a college degree or higher"])

In [123]:
master_data.head()

,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,Poverty Rate (%),Median Household Income ($),% of adults with a college degree or higher
countyFIPS,,,,,,,,,,,
01001,Autauga County,AL,2173,30,55869,3889.46,53.70,5.2,13.8,59338,27.7
01003,Baldwin County,AL,6966,71,223234,3120.49,31.81,5.5,9.8,57588,31.3
01005,Barbour County,AL,1061,9,24686,4297.98,36.46,8.6,30.9,34382,12.2
01007,Bibb County,AL,878,15,22394,3920.69,66.98,6.6,21.8,46064,11.5
01009,Blount County,AL,2095,25,57826,3622.94,43.23,4,13.2,50412,12.6


### Race and gender demographic data

In [124]:
race_data = pd.read_csv('Data/race_all.csv', encoding='latin-1')

In [125]:
race_data.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,...,607,538,57,48,26,32,9,11,19,10
1,50,1,1,Alabama,Autauga County,1,1,3579,1866,1713,...,77,56,9,5,4,1,0,0,2,1
2,50,1,1,Alabama,Autauga County,1,2,3991,2001,1990,...,64,66,2,3,2,7,2,3,2,0
3,50,1,1,Alabama,Autauga County,1,3,4290,2171,2119,...,51,57,13,7,5,5,2,1,1,1
4,50,1,1,Alabama,Autauga County,1,4,4290,2213,2077,...,48,44,7,5,0,2,2,1,3,1


#### This dataset has far more information than I need. It also does not give me the ethnic breakdown by percent, so I will have to calculate that myself. I will perform these operations below with accompanying comments.

I only want the 2019 estimates which corresponds to a YEAR value of 12. I also don't care to have my data divided by age group; I only care about totals. So I will select the AGEGRP of 0 which corresponds to all ages (I don't restrict my data to voting age because there isn't a group that splits cleanly at 18).

In [126]:
race_data = race_data[(race_data['YEAR'] == 12) & (race_data['AGEGRP'] == 0)]

In [127]:
race_data.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
209,50,1,1,Alabama,Autauga County,12,0,55869,27092,28777,...,778,687,89,93,40,27,15,19,16,11
437,50,1,3,Alabama,Baldwin County,12,0,223234,108247,114987,...,5144,4646,268,281,264,197,69,65,55,35
665,50,1,5,Alabama,Barbour County,12,0,24686,13064,11622,...,509,408,63,50,61,26,1,0,14,8
893,50,1,7,Alabama,Bibb County,12,0,22394,11929,10465,...,291,253,32,19,6,15,5,1,17,3
1121,50,1,9,Alabama,Blount County,12,0,57826,28472,29354,...,2794,2516,76,58,67,66,18,21,34,21


Next I will select the relevant columns. This data consists of raw numbers rather than percentages, so I will need the TOT_POP (total population) column to compute percentages for each category. I will include the categories of Black, White, Asian, Hispanic, Native American, and Hawaiian/Pacific Islander. The data gets tricky because Hispanic is considered an ethnicity rather than a race, so they can be of any race. This means that the percentages may add up to more than 100%. The census includes some categories that I can use to reduce overlap as much as possible. These categories include "Not Hispanic, White alone population", "Not Hispanic, Black or African American alone population", "Not Hispanic, American Indian and Alaska Native alone population", "Not Hispanic, Asian alone population", and "Not Hispanic, Native Hawaiian and Other Pacific Islander alone population". Finally there is a "Hispanic population" category which captures Hispanic combined with any race. Each of these categories is split by gender, so I will have to select both genders and create a column for totals.

One final note. There are some groups that might not identify neatly into any of these census categories. I will assume that the census handles this in their own way and instructs these individuals to classify themselves accordingly.

In [128]:
race_data.columns

Index(['SUMLEV', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'YEAR', 'AGEGRP',
       'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'WA_MALE', 'WA_FEMALE', 'BA_MALE',
       'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE',
       'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'WAC_MALE', 'WAC_FEMALE',
       'BAC_MALE', 'BAC_FEMALE', 'IAC_MALE', 'IAC_FEMALE', 'AAC_MALE',
       'AAC_FEMALE', 'NAC_MALE', 'NAC_FEMALE', 'NH_MALE', 'NH_FEMALE',
       'NHWA_MALE', 'NHWA_FEMALE', 'NHBA_MALE', 'NHBA_FEMALE', 'NHIA_MALE',
       'NHIA_FEMALE', 'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 'NHNA_FEMALE',
       'NHTOM_MALE', 'NHTOM_FEMALE', 'NHWAC_MALE', 'NHWAC_FEMALE',
       'NHBAC_MALE', 'NHBAC_FEMALE', 'NHIAC_MALE', 'NHIAC_FEMALE',
       'NHAAC_MALE', 'NHAAC_FEMALE', 'NHNAC_MALE', 'NHNAC_FEMALE', 'H_MALE',
       'H_FEMALE', 'HWA_MALE', 'HWA_FEMALE', 'HBA_MALE', 'HBA_FEMALE',
       'HIA_MALE', 'HIA_FEMALE', 'HAA_MALE', 'HAA_FEMALE', 'HNA_MALE',
       'HNA_FEMALE', 'HTOM_MALE', 'HTOM_FEMALE

In [129]:
# Select the relevant columns
race_data = race_data[['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'NHWA_MALE', 'NHWA_FEMALE', 'NHBA_MALE', 
                       'NHBA_FEMALE', 'NHIA_MALE', 'NHIA_FEMALE', 'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 
                       'NHNA_FEMALE', 'H_MALE', 'H_FEMALE']]

In [130]:
race_data.head()

,STATE,COUNTY,STNAME,CTYNAME,TOT_POP,TOT_MALE,TOT_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,NHBA_FEMALE,NHIA_MALE,NHIA_FEMALE,NHAA_MALE,NHAA_FEMALE,NHNA_MALE,NHNA_FEMALE,H_MALE,H_FEMALE
209,1,1,Alabama,Autauga County,55869,27092,28777,20138,21077,5171,5927,105,138,282,364,20,20,884,787
437,1,3,Alabama,Baldwin County,223234,108247,114987,89845,95902,9308,9907,753,754,911,1435,53,70,5545,4989
665,1,5,Alabama,Barbour County,24686,13064,11622,5894,5341,6260,5547,52,43,55,61,21,10,629,488
893,1,7,Alabama,Bibb County,22394,11929,10465,8482,8181,2912,1807,50,41,21,25,5,1,343,280
1121,1,9,Alabama,Blount County,57826,28472,29354,24494,25682,453,419,143,139,73,90,14,7,2950,2632


Create columns for the general gender breakdown of each county. This may not be an important feature for predicting who a county voted for, but it will be interesting data nonetheless.

In [131]:
# Male population
race_data['% Male'] = race_data.apply(lambda x: round((x['TOT_MALE'] / x['TOT_POP']) * 100, 2) , axis=1)

In [132]:
# Female population
race_data['% Female'] = race_data.apply(lambda x: round((x['TOT_FEMALE'] / x['TOT_POP']) * 100, 2) , axis=1)

In [133]:
race_data.head()

,STATE,COUNTY,STNAME,CTYNAME,TOT_POP,TOT_MALE,TOT_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,...,NHIA_MALE,NHIA_FEMALE,NHAA_MALE,NHAA_FEMALE,NHNA_MALE,NHNA_FEMALE,H_MALE,H_FEMALE,% Male,% Female
209,1,1,Alabama,Autauga County,55869,27092,28777,20138,21077,5171,...,105,138,282,364,20,20,884,787,48.49,51.51
437,1,3,Alabama,Baldwin County,223234,108247,114987,89845,95902,9308,...,753,754,911,1435,53,70,5545,4989,48.49,51.51
665,1,5,Alabama,Barbour County,24686,13064,11622,5894,5341,6260,...,52,43,55,61,21,10,629,488,52.92,47.08
893,1,7,Alabama,Bibb County,22394,11929,10465,8482,8181,2912,...,50,41,21,25,5,1,343,280,53.27,46.73
1121,1,9,Alabama,Blount County,57826,28472,29354,24494,25682,453,...,143,139,73,90,14,7,2950,2632,49.24,50.76


In [134]:
# Drop the original male and female population columns
race_data.drop(['TOT_MALE', 'TOT_FEMALE'], axis=1, inplace=True)

Create columns for the percentage of each race in the population.

In [135]:
def race_percentage(male_pop, female_pop, county_total_pop):
    """
    Function takes the male and female populations of a specific race, 
    sums them and calculates the percentage of the total that this 
    race comprises.
    Returns a percentage rounded to the nearest hundredth.
    """
    percent = (male_pop + female_pop) / county_total_pop
    return round(percent * 100, 2)

In [136]:
# White
race_data['% White'] = race_data.apply(lambda x: race_percentage(x['NHWA_MALE'], x['NHWA_FEMALE'], x['TOT_POP']), 
                                       axis=1)

In [137]:
# Black
race_data['% Black'] = race_data.apply(lambda x: race_percentage(x['NHBA_MALE'], x['NHBA_FEMALE'], x['TOT_POP']), 
                                       axis=1)

In [138]:
# Native American
race_data['% Native American'] = race_data.apply(lambda x: race_percentage(x['NHIA_MALE'], x['NHIA_FEMALE'], 
                                                                           x['TOT_POP']), axis=1)

In [139]:
# Asian
race_data['% Asian'] = race_data.apply(lambda x: race_percentage(x['NHAA_MALE'], x['NHAA_FEMALE'], x['TOT_POP']), 
                                       axis=1)

In [140]:
# Hawaiian/Pacific Islander
race_data['% Hawaiian or Pacific Islander'] = race_data.apply(lambda x: race_percentage(x['NHNA_MALE'], 
                                                                                        x['NHNA_FEMALE'], 
                                                                                        x['TOT_POP']), axis=1)

In [141]:
# Hispanic or Latino
race_data['% Hispanic or Latino'] = race_data.apply(lambda x: race_percentage(x['H_MALE'], x['H_FEMALE'], 
                                                                              x['TOT_POP']), axis=1)

In [142]:
race_data.head()

,STATE,COUNTY,STNAME,CTYNAME,TOT_POP,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,NHBA_FEMALE,NHIA_MALE,...,H_MALE,H_FEMALE,% Male,% Female,% White,% Black,% Native American,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino
209,1,1,Alabama,Autauga County,55869,20138,21077,5171,5927,105,...,884,787,48.49,51.51,73.77,19.86,0.43,1.16,0.07,2.99
437,1,3,Alabama,Baldwin County,223234,89845,95902,9308,9907,753,...,5545,4989,48.49,51.51,83.21,8.61,0.68,1.05,0.06,4.72
665,1,5,Alabama,Barbour County,24686,5894,5341,6260,5547,52,...,629,488,52.92,47.08,45.51,47.83,0.38,0.47,0.13,4.52
893,1,7,Alabama,Bibb County,22394,8482,8181,2912,1807,50,...,343,280,53.27,46.73,74.41,21.07,0.41,0.21,0.03,2.78
1121,1,9,Alabama,Blount County,57826,24494,25682,453,419,143,...,2950,2632,49.24,50.76,86.77,1.51,0.49,0.28,0.04,9.65


#### Drop the original columns

In [143]:
race_data.columns

Index(['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'TOT_POP', 'NHWA_MALE',
       'NHWA_FEMALE', 'NHBA_MALE', 'NHBA_FEMALE', 'NHIA_MALE', 'NHIA_FEMALE',
       'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 'NHNA_FEMALE', 'H_MALE',
       'H_FEMALE', '% Male', '% Female', '% White', '% Black',
       '% Native American', '% Asian', '% Hawaiian or Pacific Islander',
       '% Hispanic or Latino'],
      dtype='object')

In [144]:
race_data.drop(['TOT_POP', 'NHWA_MALE', 'NHWA_FEMALE', 'NHBA_MALE', 'NHBA_FEMALE', 'NHIA_MALE', 'NHIA_FEMALE', 
                'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 'NHNA_FEMALE', 'H_MALE', 'H_FEMALE'], axis=1, inplace=True)

In [145]:
race_data.head()

,STATE,COUNTY,STNAME,CTYNAME,% Male,% Female,% White,% Black,% Native American,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino
209,1,1,Alabama,Autauga County,48.49,51.51,73.77,19.86,0.43,1.16,0.07,2.99
437,1,3,Alabama,Baldwin County,48.49,51.51,83.21,8.61,0.68,1.05,0.06,4.72
665,1,5,Alabama,Barbour County,52.92,47.08,45.51,47.83,0.38,0.47,0.13,4.52
893,1,7,Alabama,Bibb County,53.27,46.73,74.41,21.07,0.41,0.21,0.03,2.78
1121,1,9,Alabama,Blount County,49.24,50.76,86.77,1.51,0.49,0.28,0.04,9.65


#### To join with master_data, race_data will need to be indexed by county FIPS codes. Below I will create a column of these codes and set the dataframe's index to these codes.

In [146]:
# Use the combine_fips function created earlier to create the county FIPS codes in a new column
race_data['countyFIPS'] = race_data.apply(lambda x: combine_fips(x['STATE'], x['COUNTY']), axis=1)

In [147]:
# Set the dataframe's index to the FIPS codes and drop the old columns
race_data.set_index('countyFIPS', inplace=True)
race_data.drop(['STATE', 'COUNTY'], axis=1, inplace=True)

In [148]:
race_data.head()

,STNAME,CTYNAME,% Male,% Female,% White,% Black,% Native American,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino
countyFIPS,,,,,,,,,,
01001,Alabama,Autauga County,48.49,51.51,73.77,19.86,0.43,1.16,0.07,2.99
01003,Alabama,Baldwin County,48.49,51.51,83.21,8.61,0.68,1.05,0.06,4.72
01005,Alabama,Barbour County,52.92,47.08,45.51,47.83,0.38,0.47,0.13,4.52
01007,Alabama,Bibb County,53.27,46.73,74.41,21.07,0.41,0.21,0.03,2.78
01009,Alabama,Blount County,49.24,50.76,86.77,1.51,0.49,0.28,0.04,9.65


In [149]:
print(race_data.shape)
print(master_data.shape)

(3142, 10)
(3141, 11)


The race_data has one more row than master_data. Check to see if race_data includes Kalawao County, HI and if so drop it from the dataframe.

In [150]:
race_data[race_data['STNAME'] == 'Hawaii']

,STNAME,CTYNAME,% Male,% Female,% White,% Black,% Native American,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino
countyFIPS,,,,,,,,,,
15001,Hawaii,Hawaii County,49.45,50.55,30.20,0.71,0.32,20.19,11.78,12.90
15003,Hawaii,Honolulu County,50.23,49.77,17.88,2.54,0.18,41.78,8.93,9.98
15005,Hawaii,Kalawao County,47.67,52.33,26.74,0.00,0.00,8.14,48.84,1.16
15007,Hawaii,Kauai County,49.38,50.62,29.28,0.61,0.26,29.54,8.40,11.36
15009,Hawaii,Maui County,49.63,50.37,30.08,0.69,0.28,27.89,9.84,11.57


In [151]:
# Drop Kalawao County from the dataframe
race_data.drop(labels='15005', inplace=True)
race_data.shape

(3141, 10)

In [152]:
# Save race data
race_data.to_csv('Data/cleaned_race_gender_data.csv')

### Join master_data and race_data

In [153]:
master_data = master_data.join(race_data[['% Male', '% Female', '% White', '% Black', '% Native American', '% Asian', 
                                          '% Hawaiian or Pacific Islander', '% Hispanic or Latino']])

In [155]:
print(master_data.shape)
master_data.head()

(3141, 19)


,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,Poverty Rate (%),Median Household Income ($),% of adults with a college degree or higher,% Male,% Female,% White,% Black,% Native American,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino
countyFIPS,,,,,,,,,,,,,,,,,,,
01001,Autauga County,AL,2173,30,55869,3889.46,53.70,5.2,13.8,59338,27.7,48.49,51.51,73.77,19.86,0.43,1.16,0.07,2.99
01003,Baldwin County,AL,6966,71,223234,3120.49,31.81,5.5,9.8,57588,31.3,48.49,51.51,83.21,8.61,0.68,1.05,0.06,4.72
01005,Barbour County,AL,1061,9,24686,4297.98,36.46,8.6,30.9,34382,12.2,52.92,47.08,45.51,47.83,0.38,0.47,0.13,4.52
01007,Bibb County,AL,878,15,22394,3920.69,66.98,6.6,21.8,46064,11.5,53.27,46.73,74.41,21.07,0.41,0.21,0.03,2.78
01009,Blount County,AL,2095,25,57826,3622.94,43.23,4,13.2,50412,12.6,49.24,50.76,86.77,1.51,0.49,0.28,0.04,9.65


## Population Density

I was having trouble finding population density figures for the most recent population estimates, so I will have to calculate these figures myself. To calculate population density, I will simply divide a county's total population by the county's land area measured in square miles. First I need the land area of each county.

In [156]:
land_area = pd.read_excel('Data/land_area.xls')

In [157]:
land_area

,Areaname,STCOU,LND010190F,LND010190D,LND010190N1,LND010190N2,LND010200F,LND010200D,LND010200N1,LND010200N2,...,LND110210N1,LND110210N2,LND210190F,LND210190D,LND210190N1,LND210190N2,LND210200F,LND210200D,LND210200N1,LND210200N2
0,UNITED STATES,0,0,3787425.08,0,0,0,3794083.06,0,0,...,0,0,0,251083.35,0,0,0,256644.62,0,0
1,ALABAMA,1000,0,52422.94,0,0,0,52419.02,0,0,...,0,0,0,1672.71,0,0,0,1675.01,0,0
2,"Autauga, AL",1001,0,604.49,0,0,0,604.45,0,0,...,0,0,0,8.48,0,0,0,8.48,0,0
3,"Baldwin, AL",1003,0,2027.08,0,0,0,2026.93,0,0,...,0,0,0,430.55,0,0,0,430.58,0,0
4,"Barbour, AL",1005,0,904.59,0,0,0,904.52,0,0,...,0,0,0,19.59,0,0,0,19.61,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3193,"Sweetwater, WY",56037,0,10491.73,0,0,0,10491.17,0,0,...,0,0,0,65.86,0,0,0,65.87,0,0
3194,"Teton, WY",56039,0,4221.96,0,0,0,4221.80,0,0,...,0,0,0,214.02,0,0,0,214.04,0,0
3195,"Uinta, WY",56041,0,2087.66,0,0,0,2087.56,0,0,...,0,0,0,5.91,0,0,0,5.90,0,0
3196,"Washakie, WY",56043,0,2242.85,0,0,0,2242.75,0,0,...,0,0,0,2.69,0,0,0,2.69,0,0


This dataframe uses a special encoding as its columns names. The one I want is 'LND110210D'. I will select this and other relevant columns.

In [158]:
land_area = land_area[['Areaname', 'STCOU', 'LND110210D']]

In [159]:
print(land_area.shape)
land_area.head()

(3198, 3)


,Areaname,STCOU,LND110210D
0,UNITED STATES,0,3531905.43
1,ALABAMA,1000,50645.33
2,"Autauga, AL",1001,594.44
3,"Baldwin, AL",1003,1589.78
4,"Barbour, AL",1005,884.88


Remove the non-county rows.

In [161]:
# First properly format the FIPS codes
land_area['STCOU'] = land_area['STCOU'].apply(pad_fips)

In [162]:
# Remove the non-county rows
land_area = land_area[land_area['STCOU'].apply(lambda code: code[2:] != '000')]

In [163]:
print(land_area.shape)
land_area.head()

(3146, 3)


,Areaname,STCOU,LND110210D
2,"Autauga, AL",01001,594.44
3,"Baldwin, AL",01003,1589.78
4,"Barbour, AL",01005,884.88
5,"Bibb, AL",01007,622.58
6,"Blount, AL",01009,644.78


Set this dataframe's index to the FIPS codes.

In [164]:
# First rename the columns
land_area.rename(columns={'STCOU': 'countyFIPS', 'LND110210D': 'Land Area (sq. mi)'}, inplace=True)

In [165]:
# Set the index to the FIPS codes
land_area.set_index('countyFIPS', inplace=True)

In [167]:
land_area

,Areaname,Land Area (sq. mi)
countyFIPS,,
01001,"Autauga, AL",594.44
01003,"Baldwin, AL",1589.78
01005,"Barbour, AL",884.88
01007,"Bibb, AL",622.58
01009,"Blount, AL",644.78
...,...,...
56037,"Sweetwater, WY",10426.65
56039,"Teton, WY",3995.38
56041,"Uinta, WY",2081.26


This dataframe has a few extra rows. I will join this dataframe with master_data as a test to view the extra rows.

In [168]:
test_df = land_area.join(master_data, how='outer', lsuffix='_land', rsuffix='_master')

In [169]:
test_df[test_df.isna()['COVID Cases']]

,Areaname,Land Area (sq. mi),County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,...,Median Household Income ($),% of adults with a college degree or higher,% Male,% Female,% White,% Black,% Native American,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino
countyFIPS,,,,,,,,,,,,,,,,,,,,,
02270,"Wade Hampton, AK",17081.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15005,"Kalawao, HI",11.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30113,"Yellowstone National Park, MT",0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46113,"Shannon, SD",2093.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51515,"Bedford, VA",6.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51560,"Clifton Forge, VA",0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51780,"South Boston, VA",0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


land_area had 5 more rows than master_data, but it turns out that land_area actually has 7 rows that master_data does not have. This means that if I remove these rows, land_area will be missing 2 counties that appear in master_data. I will check to see which counties are in master_data but missing from land_area.

In [170]:
test_df[test_df.isna()['Land Area (sq. mi)']]

,Areaname,Land Area (sq. mi),County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,...,Median Household Income ($),% of adults with a college degree or higher,% Male,% Female,% White,% Black,% Native American,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino
countyFIPS,,,,,,,,,,,,,,,,,,,,,
02158,NaN,NaN,Kusilvak Census Area,AK,250.0,0.0,8314.0,3006.98,0.00,13.2,...,32728,4.8,52.66,47.34,3.38,0.38,90.41,0.42,0.02,2.99
46102,NaN,NaN,Oglala Lakota County,SD,1051.0,9.0,14177.0,7413.42,63.48,10.3,...,31662,11.8,49.10,50.90,4.72,0.39,89.21,0.13,0.06,4.08


After investigating a little, it turns out that Kusilvak Census Area in Alaska used to be called Wade Hampton Census Area and Oglala Lakota County in South Dakota used to be called Shannon County. So both datasets have these counties, they just go by different names and FIPS codes. I will change the FIPS code of these two counties in land_area so they map properly when I join master_data with land_area.

In [171]:
# Get the dataframe's index as a list, find the index of the counties of interest
as_list = land_area.index.tolist()
wade_idx = as_list.index('02270')
shannon_idx = as_list.index('46113')

In [172]:
# Change the old FIPS codes to the new ones that occur in master_data
as_list[wade_idx] = '02158'
as_list[shannon_idx] = '46102'

In [173]:
# Set land_area's index to the list containing the new county indexes
land_area.index = as_list

Now remove the rows that do not appear in master_data that were returned when you calculated test_df initially.

In [174]:
test_df[test_df.isna()['COVID Cases']].index

Index(['02270', '15005', '30113', '46113', '51515', '51560', '51780'], dtype='object', name='countyFIPS')

In [175]:
land_area.drop(['15005', '30113', '51515', '51560', '51780'], inplace=True)

In [176]:
print(land_area.shape)
land_area.head()

(3141, 2)


,Areaname,Land Area (sq. mi)
01001,"Autauga, AL",594.44
01003,"Baldwin, AL",1589.78
01005,"Barbour, AL",884.88
01007,"Bibb, AL",622.58
01009,"Blount, AL",644.78


In [177]:
# Save dataframe
land_area.to_csv('Data/cleaned_land_area_data.csv')

### Now join master_data and land_area

In [178]:
master_data = master_data.join(land_area['Land Area (sq. mi)'])

#### Calculate population density using the land area and population columns

In [179]:
import math

In [180]:
# Use math.ceil to round up so you don't end up with a county showing a population density of 0 unless the 
# population of that county actually is 0.
master_data['Population Density (per sq. mi)'] = master_data.apply(
    lambda x: math.ceil(x['Population'] / x['Land Area (sq. mi)']), axis=1)

In [181]:
# Drop the land area column because I only needed it to calculate population density
master_data.drop('Land Area (sq. mi)', axis=1, inplace=True)

In [182]:
master_data.head()

,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,Poverty Rate (%),Median Household Income ($),% of adults with a college degree or higher,% Male,% Female,% White,% Black,% Native American,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino,Population Density (per sq. mi)
countyFIPS,,,,,,,,,,,,,,,,,,,,
01001,Autauga County,AL,2173,30,55869,3889.46,53.70,5.2,13.8,59338,27.7,48.49,51.51,73.77,19.86,0.43,1.16,0.07,2.99,94
01003,Baldwin County,AL,6966,71,223234,3120.49,31.81,5.5,9.8,57588,31.3,48.49,51.51,83.21,8.61,0.68,1.05,0.06,4.72,141
01005,Barbour County,AL,1061,9,24686,4297.98,36.46,8.6,30.9,34382,12.2,52.92,47.08,45.51,47.83,0.38,0.47,0.13,4.52,28
01007,Bibb County,AL,878,15,22394,3920.69,66.98,6.6,21.8,46064,11.5,53.27,46.73,74.41,21.07,0.41,0.21,0.03,2.78,36
01009,Blount County,AL,2095,25,57826,3622.94,43.23,4,13.2,50412,12.6,49.24,50.76,86.77,1.51,0.49,0.28,0.04,9.65,90


### Median Age

In [183]:
median_age = pd.read_csv('Data/median_age.csv', header=1)

In [184]:
median_age.head()

,id,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total MOE!!Total population,Estimate!!Percent!!Total population,Margin of Error!!Percent MOE!!Total population,Estimate!!Male!!Total population,Margin of Error!!Male MOE!!Total population,Estimate!!Percent Male!!Total population,Margin of Error!!Percent Male MOE!!Total population,...,Estimate!!Percent!!PERCENT ALLOCATED!!Age,Margin of Error!!Percent MOE!!PERCENT ALLOCATED!!Age,Estimate!!Male!!PERCENT ALLOCATED!!Age,Margin of Error!!Male MOE!!PERCENT ALLOCATED!!Age,Estimate!!Percent Male!!PERCENT ALLOCATED!!Age,Margin of Error!!Percent Male MOE!!PERCENT ALLOCATED!!Age,Estimate!!Female!!PERCENT ALLOCATED!!Age,Margin of Error!!Female MOE!!PERCENT ALLOCATED!!Age,Estimate!!Percent Female!!PERCENT ALLOCATED!!Age,Margin of Error!!Percent Female MOE!!PERCENT ALLOCATED!!Age
0,0500000US01001,"Autauga County, Alabama",55200,*****,(X),(X),26874,147,(X),(X),...,1.3,(X),(X),(X),(X),(X),(X),(X),(X),(X)
1,0500000US01003,"Baldwin County, Alabama",208107,*****,(X),(X),101188,242,(X),(X),...,1.8,(X),(X),(X),(X),(X),(X),(X),(X),(X)
2,0500000US01005,"Barbour County, Alabama",25782,*****,(X),(X),13697,71,(X),(X),...,0.9,(X),(X),(X),(X),(X),(X),(X),(X),(X)
3,0500000US01007,"Bibb County, Alabama",22527,*****,(X),(X),12152,180,(X),(X),...,8.2,(X),(X),(X),(X),(X),(X),(X),(X),(X)
4,0500000US01009,"Blount County, Alabama",57645,*****,(X),(X),28434,137,(X),(X),...,1.0,(X),(X),(X),(X),(X),(X),(X),(X),(X)


In [185]:
# Split 'Geographic Area Name' into County Name and State columns
median_age['County Name'] = median_age['Geographic Area Name'].apply(lambda name: name.split(',')[0].strip())
median_age['State'] = median_age['Geographic Area Name'].apply(lambda name: name.split(',')[-1].strip())

In [186]:
# Drop the original 'Geographic Area Name' column
median_age.drop('Geographic Area Name', axis=1, inplace=True)

There are more columns in this dataset than we need and the names are very messy. I'm looking for the median age of the population in each county, so I will search for column names that include the word "Median".

In [187]:
median_age[[name for name in median_age.columns if 'Median' in name]]

,Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years),Margin of Error!!Total MOE!!Total population!!SUMMARY INDICATORS!!Median age (years),Estimate!!Percent!!Total population!!SUMMARY INDICATORS!!Median age (years),Margin of Error!!Percent MOE!!Total population!!SUMMARY INDICATORS!!Median age (years),Estimate!!Male!!Total population!!SUMMARY INDICATORS!!Median age (years),Margin of Error!!Male MOE!!Total population!!SUMMARY INDICATORS!!Median age (years),Estimate!!Percent Male!!Total population!!SUMMARY INDICATORS!!Median age (years),Margin of Error!!Percent Male MOE!!Total population!!SUMMARY INDICATORS!!Median age (years),Estimate!!Female!!Total population!!SUMMARY INDICATORS!!Median age (years),Margin of Error!!Female MOE!!Total population!!SUMMARY INDICATORS!!Median age (years),Estimate!!Percent Female!!Total population!!SUMMARY INDICATORS!!Median age (years),Margin of Error!!Percent Female MOE!!Total population!!SUMMARY INDICATORS!!Median age (years)
0,37.8,0.4,(X),(X),36.9,0.6,(X),(X),38.9,0.8,(X),(X)
1,42.8,0.3,(X),(X),41.8,0.4,(X),(X),44.1,0.3,(X),(X)
2,39.9,0.5,(X),(X),38.1,0.7,(X),(X),43.4,0.5,(X),(X)
3,39.9,1.1,(X),(X),37.5,1.2,(X),(X),43.1,1.4,(X),(X)
4,40.8,0.4,(X),(X),40.2,0.6,(X),(X),41.7,0.7,(X),(X)
...,...,...,...,...,...,...,...,...,...,...,...,...
3215,40.7,0.4,(X),(X),39.2,0.7,(X),(X),42.1,0.5,(X),(X)
3216,43.6,0.5,(X),(X),40.8,7.5,(X),(X),45.0,1.6,(X),(X)
3217,38.8,0.9,(X),(X),36.9,0.8,(X),(X),40.3,0.7,(X),(X)
3218,42.5,0.4,(X),(X),39.9,2.2,(X),(X),43.7,0.8,(X),(X)


The column with the name "Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)" is the one I want, so I will select this column from median_age along with other relevant columns.

In [188]:
median_age = median_age[['id', 'State', 'County Name', 
            'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)']]

In [189]:
median_age.head()

,id,State,County Name,Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)
0,0500000US01001,Alabama,Autauga County,37.8
1,0500000US01003,Alabama,Baldwin County,42.8
2,0500000US01005,Alabama,Barbour County,39.9
3,0500000US01007,Alabama,Bibb County,39.9
4,0500000US01009,Alabama,Blount County,40.8


In [190]:
# Rename columns
median_age.rename(columns={'id': 'countyFIPS', 
                   'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)': 'Median Age'}, 
                  inplace=True)

In [192]:
median_age

,countyFIPS,State,County Name,Median Age
0,0500000US01001,Alabama,Autauga County,37.8
1,0500000US01003,Alabama,Baldwin County,42.8
2,0500000US01005,Alabama,Barbour County,39.9
3,0500000US01007,Alabama,Bibb County,39.9
4,0500000US01009,Alabama,Blount County,40.8
...,...,...,...,...
3215,0500000US72145,Puerto Rico,Vega Baja Municipio,40.7
3216,0500000US72147,Puerto Rico,Vieques Municipio,43.6
3217,0500000US72149,Puerto Rico,Villalba Municipio,38.8
3218,0500000US72151,Puerto Rico,Yabucoa Municipio,42.5


#### Remove Puerto Rico counties

In [193]:
median_age = median_age[median_age['State'] != 'Puerto Rico']

In [194]:
median_age.shape

(3142, 4)

This median_age data frame has 1 more row than master_data. Chances are it is Kalawao County, HI. Which I will remove because I don't have voting data on that county.

In [195]:
median_age[median_age['State'] == 'Hawaii']

,countyFIPS,State,County Name,Median Age
546,0500000US15001,Hawaii,Hawaii County,42.3
547,0500000US15003,Hawaii,Honolulu County,37.6
548,0500000US15005,Hawaii,Kalawao County,57.1
549,0500000US15007,Hawaii,Kauai County,42.4
550,0500000US15009,Hawaii,Maui County,41.1


In [197]:
median_age.drop(labels=548, inplace=True)
print(median_age.shape)

(3141, 4)


#### Reformat the countyFIPS column and set this column to be the dataframe's index.

In [199]:
median_age['countyFIPS'] = median_age['countyFIPS'].apply(lambda code: code[-5:])

In [200]:
median_age.set_index('countyFIPS', inplace=True)

In [201]:
median_age.head()

,State,County Name,Median Age
countyFIPS,,,
01001,Alabama,Autauga County,37.8
01003,Alabama,Baldwin County,42.8
01005,Alabama,Barbour County,39.9
01007,Alabama,Bibb County,39.9
01009,Alabama,Blount County,40.8


In [202]:
# Save median_age data
median_age.to_csv('Data/cleaned_median_age_data.csv')

### Join master_data and median_age

In [203]:
master_data = master_data.join(median_age['Median Age'])

### Veterans data

For each county I will calculate the percentage of the adult population that served in the military.

In [204]:
veterans_data = pd.read_csv('Data/veterans_data.csv', header=1)

/Users/PayPay/ml/my_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (20,21,32,33,44,45,56,57,68,69,80,81,92,93,104,105,116,117,128,129,140,141,152,153,164,165,176,177,188,189,200,201,212,213,224,225,236,237,248,249,260,261,266,267,274,275,320,321,332,333,344,345,356,357,362,363,366,367,370,371,376,377,384,385,386,387,390,391,394,395,400,401,408,409,410,411,414,415,418,419,422,423,424,425,426,427,430,431,432,433,434,435,436,437,438,439,442,443,444,445,464,465,476,477) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [205]:
veterans_data.head()

,id,Geographic Area Name,Estimate!!Total!!Civilian population 18 years and over,Margin of Error!!Total MOE!!Civilian population 18 years and over,Estimate!!Percent!!Civilian population 18 years and over,Margin of Error!!Percent MOE!!Civilian population 18 years and over,Estimate!!Veterans!!Civilian population 18 years and over,Margin of Error!!Veterans MOE!!Civilian population 18 years and over,Estimate!!Percent Veterans!!Civilian population 18 years and over,Margin of Error!!Percent Veterans MOE!!Civilian population 18 years and over,...,Estimate!!Percent!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Margin of Error!!Percent MOE!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Estimate!!Veterans!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Margin of Error!!Veterans MOE!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Estimate!!Percent Veterans!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Margin of Error!!Percent Veterans MOE!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Estimate!!Nonveterans!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Margin of Error!!Nonveterans MOE!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Estimate!!Percent Nonveterans!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Margin of Error!!Percent Nonveterans MOE!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability
0,0500000US01001,"Autauga County, Alabama",41284,197,(X),(X),5071,473,12.3,1.1,...,77.4,1.5,3394,433,67.4,4.6,28254,669,78.8,1.6
1,0500000US01003,"Baldwin County, Alabama",162122,149,(X),(X),19354,912,11.9,0.6,...,83.0,0.7,13169,821,69.2,2.4,119159,1219,84.8,0.7
2,0500000US01005,"Barbour County, Alabama",20346,38,(X),(X),1561,170,7.7,0.8,...,73.6,1.9,771,127,54.8,6.2,12065,403,75.2,2.1
3,0500000US01007,"Bibb County, Alabama",17868,4,(X),(X),1278,245,7.2,1.4,...,79.6,2.5,796,218,65.7,10.2,11826,478,80.8,2.7
4,0500000US01009,"Blount County, Alabama",44126,70,(X),(X),3765,320,8.5,0.7,...,82.2,1.3,2392,294,63.9,5.3,33531,573,83.9,1.3


This dataframe has many columns, but the ones I'm interested in are the estimate of the number of veterans and the estimate of the total civilian population over 18 years old. I will use these two columns to calculate the percentage of adults that are veterans. I will use the total civilian population over 18 years old to calculate this percentage rather than the county's total population because minors are not allowed to serve in the military. By considering only the adult civilian population, I calculate a metric that more accurately captures the idea behind this statistic which is "Of those who could serve in the military, how many do/did serve?"

In [206]:
# Create a new column called "% Veteran" by dividing the veteran population by the civilian population over 18
veterans_data['% Veteran'] = veterans_data.apply(
    lambda x: round((x['Estimate!!Veterans!!Civilian population 18 years and over'] / 
                     x['Estimate!!Total!!Civilian population 18 years and over']) * 100, 2), axis=1)

In [207]:
veterans_data.head()

,id,Geographic Area Name,Estimate!!Total!!Civilian population 18 years and over,Margin of Error!!Total MOE!!Civilian population 18 years and over,Estimate!!Percent!!Civilian population 18 years and over,Margin of Error!!Percent MOE!!Civilian population 18 years and over,Estimate!!Veterans!!Civilian population 18 years and over,Margin of Error!!Veterans MOE!!Civilian population 18 years and over,Estimate!!Percent Veterans!!Civilian population 18 years and over,Margin of Error!!Percent Veterans MOE!!Civilian population 18 years and over,...,Margin of Error!!Percent MOE!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Estimate!!Veterans!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Margin of Error!!Veterans MOE!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Estimate!!Percent Veterans!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Margin of Error!!Percent Veterans MOE!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Estimate!!Nonveterans!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Margin of Error!!Nonveterans MOE!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Estimate!!Percent Nonveterans!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,Margin of Error!!Percent Nonveterans MOE!!DISABILITY STATUS!!Civilian population 18 years and over for whom poverty status is determined!!Without a disability,% Veteran
0,0500000US01001,"Autauga County, Alabama",41284,197,(X),(X),5071,473,12.3,1.1,...,1.5,3394,433,67.4,4.6,28254,669,78.8,1.6,12.28
1,0500000US01003,"Baldwin County, Alabama",162122,149,(X),(X),19354,912,11.9,0.6,...,0.7,13169,821,69.2,2.4,119159,1219,84.8,0.7,11.94
2,0500000US01005,"Barbour County, Alabama",20346,38,(X),(X),1561,170,7.7,0.8,...,1.9,771,127,54.8,6.2,12065,403,75.2,2.1,7.67
3,0500000US01007,"Bibb County, Alabama",17868,4,(X),(X),1278,245,7.2,1.4,...,2.5,796,218,65.7,10.2,11826,478,80.8,2.7,7.15
4,0500000US01009,"Blount County, Alabama",44126,70,(X),(X),3765,320,8.5,0.7,...,1.3,2392,294,63.9,5.3,33531,573,83.9,1.3,8.53


In [208]:
# Split 'Geographic Area Name' into County Name and State columns
veterans_data['County Name'] = veterans_data['Geographic Area Name'].apply(lambda name: name.split(',')[0].strip())
veterans_data['State'] = veterans_data['Geographic Area Name'].apply(lambda name: name.split(',')[-1].strip())

In [209]:
# Drop the original 'Geographic Area Name' column
veterans_data.drop('Geographic Area Name', axis=1, inplace=True)

Select the relevant columns

In [210]:
veterans_data = veterans_data[['id', 'State', 'County Name', '% Veteran']]

In [211]:
veterans_data.head()

,id,State,County Name,% Veteran
0,0500000US01001,Alabama,Autauga County,12.28
1,0500000US01003,Alabama,Baldwin County,11.94
2,0500000US01005,Alabama,Barbour County,7.67
3,0500000US01007,Alabama,Bibb County,7.15
4,0500000US01009,Alabama,Blount County,8.53


Rename the FIPS code column, reformat it, and set it as the dataframe's index

In [212]:
veterans_data.rename(columns={'id': 'countyFIPS'}, inplace=True)

In [213]:
# Reformat the FIPS codes
veterans_data['countyFIPS'] = veterans_data['countyFIPS'].apply(lambda code: code[-5:])

In [214]:
# Set the index to countyFIPS
veterans_data.set_index('countyFIPS', inplace=True)

In [215]:
veterans_data

,State,County Name,% Veteran
countyFIPS,,,
01001,Alabama,Autauga County,12.28
01003,Alabama,Baldwin County,11.94
01005,Alabama,Barbour County,7.67
01007,Alabama,Bibb County,7.15
01009,Alabama,Blount County,8.53
...,...,...,...
72145,Puerto Rico,Vega Baja Municipio,2.34
72147,Puerto Rico,Vieques Municipio,6.00
72149,Puerto Rico,Villalba Municipio,2.53


Remove Puerto Rico and Kalawao County, HI

In [216]:
# Remove Puerto Rico counties
veterans_data = veterans_data[veterans_data['State'] != 'Puerto Rico']

In [218]:
# Remove Kalawao County
veterans_data.drop(labels='15005', inplace=True)

In [219]:
veterans_data.shape

(3141, 3)

In [220]:
# Save veteran data
veterans_data.to_csv('Data/cleaned_veterans_data.csv')

### Join master_data with veterans_data

In [221]:
master_data = master_data.join(veterans_data['% Veteran'])

In [222]:
master_data.head()

,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,Poverty Rate (%),Median Household Income ($),...,% Female,% White,% Black,% Native American,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino,Population Density (per sq. mi),Median Age,% Veteran
countyFIPS,,,,,,,,,,,,,,,,,,,,,
01001,Autauga County,AL,2173,30,55869,3889.46,53.70,5.2,13.8,59338,...,51.51,73.77,19.86,0.43,1.16,0.07,2.99,94,37.8,12.28
01003,Baldwin County,AL,6966,71,223234,3120.49,31.81,5.5,9.8,57588,...,51.51,83.21,8.61,0.68,1.05,0.06,4.72,141,42.8,11.94
01005,Barbour County,AL,1061,9,24686,4297.98,36.46,8.6,30.9,34382,...,47.08,45.51,47.83,0.38,0.47,0.13,4.52,28,39.9,7.67
01007,Bibb County,AL,878,15,22394,3920.69,66.98,6.6,21.8,46064,...,46.73,74.41,21.07,0.41,0.21,0.03,2.78,36,39.9,7.15
01009,Blount County,AL,2095,25,57826,3622.94,43.23,4,13.2,50412,...,50.76,86.77,1.51,0.49,0.28,0.04,9.65,90,40.8,8.53


### Population growth/decline data

The data I want to capture here is the growth or decline of each county's population over the past decade. This dataset includes US Census estimates of the population in every year from 2010-2019. I will use the 2010 and 2019 population figures to calculate the percent change in population since 2010.

In [223]:
pop_change = pd.read_csv('Data/population_change.csv', encoding='latin-1')

In [224]:
pop_change.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,40,3,6,1,0,Alabama,Alabama,4779736,4780125,4785437,...,1.917501,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,4.847310,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,24.017829,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,-5.690302,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,1.385134,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952


#### Create a new column called "Population Change (%)" using the 2010 and 2019 census estimates

In [225]:
def pct_change(start_val, end_val):
    """
    Function takes a start_val and an end_val and returns the 
    percentage change.
    """
    return round(((end_val - start_val) / start_val) * 100, 2)

In [226]:
# Calculate pct_change using the "ESTIMATESBASE2010" and "POPESTIMATE2019" columns
pop_change['Population Change (%)'] = pop_change.apply(lambda x: pct_change(x['ESTIMATESBASE2010'], 
                                                                            x['POPESTIMATE2019']), axis=1)

In [227]:
pop_change.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019,Population Change (%)
0,40,3,6,1,0,Alabama,Alabama,4779736,4780125,4785437,...,0.578434,1.186314,1.522549,0.563489,0.626357,0.745172,1.090366,1.773786,2.483744,2.57
1,50,3,6,1,1,Alabama,Autauga County,54571,54597,54773,...,6.018182,-6.226119,-3.902226,1.970443,-1.712875,4.777171,0.849656,0.540916,4.560062,2.33
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183112,...,16.641870,17.488579,22.751474,20.184334,17.725964,21.279291,22.398256,24.727215,24.380567,22.48
3,50,3,6,1,5,Alabama,Barbour County,27457,27455,27327,...,0.292676,-6.897817,-8.132185,-5.140431,-15.724575,-18.238016,-24.998528,-8.754922,-5.165664,-10.09
4,50,3,6,1,7,Alabama,Bibb County,22915,22915,22870,...,-4.998356,-3.787545,-5.797999,1.331144,1.329817,-0.708717,-3.234669,-6.857092,1.831952,-2.27


Extract the relevant columns

In [228]:
pop_change = pop_change[['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'Population Change (%)']]

In [229]:
pop_change.head()

,STATE,COUNTY,STNAME,CTYNAME,Population Change (%)
0,1,0,Alabama,Alabama,2.57
1,1,1,Alabama,Autauga County,2.33
2,1,3,Alabama,Baldwin County,22.48
3,1,5,Alabama,Barbour County,-10.09
4,1,7,Alabama,Bibb County,-2.27


In [230]:
# Create the FIPS codes out of the STATE and COUNTY columns
pop_change['countyFIPS'] = pop_change.apply(lambda x: combine_fips(x['STATE'], x['COUNTY']), axis=1)

In [231]:
# Drop the STATE and COUNTY columns
pop_change.drop(['STATE', 'COUNTY'], axis=1, inplace=True)

In [232]:
# Rename columns
pop_change.rename(columns={'STNAME': 'State', 'CTYNAME': 'County Name'}, inplace=True)

In [233]:
print(pop_change.shape)
pop_change.head()

(3193, 4)


,State,County Name,Population Change (%),countyFIPS
0,Alabama,Alabama,2.57,01000
1,Alabama,Autauga County,2.33,01001
2,Alabama,Baldwin County,22.48,01003
3,Alabama,Barbour County,-10.09,01005
4,Alabama,Bibb County,-2.27,01007


Remove the state level data

In [234]:
pop_change = pop_change[pop_change['countyFIPS'].apply(lambda code: code[2:] != '000')]

Set 'countyFIPS' to be the dataframe's index

In [235]:
pop_change.set_index('countyFIPS', inplace=True)

In [236]:
pop_change.shape

(3142, 3)

In [237]:
# Remove the extra county which is Kalawao County
pop_change.drop(labels='15005', inplace=True)

In [238]:
print(pop_change.shape)
pop_change.head()

(3141, 3)


,State,County Name,Population Change (%)
countyFIPS,,,
01001,Alabama,Autauga County,2.33
01003,Alabama,Baldwin County,22.48
01005,Alabama,Barbour County,-10.09
01007,Alabama,Bibb County,-2.27
01009,Alabama,Blount County,0.88


In [239]:
# Save pop_change dataset
pop_change.to_csv('Data/cleaned_population_change_data.csv')

### Join master_data with pop_change

In [240]:
master_data = master_data.join(pop_change['Population Change (%)'])

In [242]:
master_data.head()

,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,Poverty Rate (%),Median Household Income ($),...,% White,% Black,% Native American,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino,Population Density (per sq. mi),Median Age,% Veteran,Population Change (%)
countyFIPS,,,,,,,,,,,,,,,,,,,,,
01001,Autauga County,AL,2173,30,55869,3889.46,53.70,5.2,13.8,59338,...,73.77,19.86,0.43,1.16,0.07,2.99,94,37.8,12.28,2.33
01003,Baldwin County,AL,6966,71,223234,3120.49,31.81,5.5,9.8,57588,...,83.21,8.61,0.68,1.05,0.06,4.72,141,42.8,11.94,22.48
01005,Barbour County,AL,1061,9,24686,4297.98,36.46,8.6,30.9,34382,...,45.51,47.83,0.38,0.47,0.13,4.52,28,39.9,7.67,-10.09
01007,Bibb County,AL,878,15,22394,3920.69,66.98,6.6,21.8,46064,...,74.41,21.07,0.41,0.21,0.03,2.78,36,39.9,7.15,-2.27
01009,Blount County,AL,2095,25,57826,3622.94,43.23,4,13.2,50412,...,86.77,1.51,0.49,0.28,0.04,9.65,90,40.8,8.53,0.88


### Now code each county as a Biden or Trump vote

I will use a binary variable to classify whether a county voted for Biden or Trump. Biden counties will be coded "1" and Trump counties "0". This means that the degree to which a candidate won a county will not be reflected in my dataset. I am only concerned with who received more votes.

To code these counties, I'm using a dataset of county level 2020 election results with data scraped from Fox News, Politico, and The New York Times. Unfortunately, the Alaska returns are given by electoral district rather than county, so voting data for Alaskan counties will be missing. I will still include Alaska in my data visualization mapping of the counties, but it will not be considered when I build my predictive model.

Finally, it's important to note that votes are still being counted as of this writing (11/13/20), so if there are counties that are so close that they flip to the other candidate, it will not be reflected in my current data. Fortunately the vast majority of counties have either counted all of the votes or are not close enough to flip.

In [243]:
election_results = pd.read_csv('Data/2020_election_county_results.csv')

In [244]:
election_results

,state_name,county_fips,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
0,Alabama,1001,Autauga County,19764,7450,27639,12314,0.715077,0.269547,0.445530
1,Alabama,1003,Baldwin County,83055,24344,108945,58711,0.762357,0.223452,0.538905
2,Alabama,1005,Barbour County,5605,4772,10457,833,0.536005,0.456345,0.079660
3,Alabama,1007,Bibb County,7508,1982,9573,5526,0.784289,0.207041,0.577249
4,Alabama,1009,Blount County,24595,2627,27459,21968,0.895699,0.095670,0.800029
...,...,...,...,...,...,...,...,...,...,...
3154,Wyoming,56037,Sweetwater County,12197,3822,16489,8375,0.739705,0.231791,0.507914
3155,Wyoming,56039,Teton County,4341,9848,14677,-5507,0.295769,0.670982,-0.375213
3156,Wyoming,56041,Uinta County,7494,1591,9400,5903,0.797234,0.169255,0.627979
3157,Wyoming,56043,Washakie County,3245,651,4012,2594,0.808824,0.162263,0.646560


#### Create a new column with a binary variable for who won each county. 1 for Biden, 0 for Trump.

In [245]:
election_results['Biden_or_Trump'] = election_results.apply(lambda x: int(x['votes_dem'] > x['votes_gop']), axis=1)

Reformat FIPS codes and set FIPS codes as the dataframe's index

In [246]:
election_results['county_fips'] = election_results['county_fips'].apply(pad_fips)

In [247]:
# Rename coulmn to match other dataframes
election_results.rename(columns={'county_fips': 'countyFIPS'}, inplace=True)

In [248]:
# Set countyFIPS to be the index of the dataframe
election_results.set_index('countyFIPS', inplace=True)

In [249]:
print(election_results.shape)
election_results.head()

(3159, 10)


,state_name,county_name,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff,Biden_or_Trump
countyFIPS,,,,,,,,,,
01001,Alabama,Autauga County,19764,7450,27639,12314,0.715077,0.269547,0.445530,0
01003,Alabama,Baldwin County,83055,24344,108945,58711,0.762357,0.223452,0.538905,0
01005,Alabama,Barbour County,5605,4772,10457,833,0.536005,0.456345,0.079660,0
01007,Alabama,Bibb County,7508,1982,9573,5526,0.784289,0.207041,0.577249,0
01009,Alabama,Blount County,24595,2627,27459,21968,0.895699,0.095670,0.800029,0


In [250]:
# Save election dataset
election_results.to_csv('Data/cleaned_election_results.csv')

### Join master_data with election_results

Include the binary Biden_or_Trump column as well as the actual percentages for each candidate.

In [251]:
master_data = master_data.join(election_results[['per_gop', 'per_dem', 'Biden_or_Trump']])

In [252]:
# Rename the per_gop and per_dem columns
master_data.rename(columns={'per_gop': 'Trump %', 'per_dem': 'Biden %'}, inplace=True)

In [253]:
print(master_data.shape)
master_data.head()

(3141, 26)


,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,Poverty Rate (%),Median Household Income ($),...,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino,Population Density (per sq. mi),Median Age,% Veteran,Population Change (%),Trump %,Biden %,Biden_or_Trump
countyFIPS,,,,,,,,,,,,,,,,,,,,,
01001,Autauga County,AL,2173,30,55869,3889.46,53.70,5.2,13.8,59338,...,1.16,0.07,2.99,94,37.8,12.28,2.33,0.715077,0.269547,0.0
01003,Baldwin County,AL,6966,71,223234,3120.49,31.81,5.5,9.8,57588,...,1.05,0.06,4.72,141,42.8,11.94,22.48,0.762357,0.223452,0.0
01005,Barbour County,AL,1061,9,24686,4297.98,36.46,8.6,30.9,34382,...,0.47,0.13,4.52,28,39.9,7.67,-10.09,0.536005,0.456345,0.0
01007,Bibb County,AL,878,15,22394,3920.69,66.98,6.6,21.8,46064,...,0.21,0.03,2.78,36,39.9,7.15,-2.27,0.784289,0.207041,0.0
01009,Blount County,AL,2095,25,57826,3622.94,43.23,4,13.2,50412,...,0.28,0.04,9.65,90,40.8,8.53,0.88,0.895699,0.095670,0.0


#### I now have all the data I need for my visualizations and model. However, first I need to make sure all the featues have the correct data type.

In [254]:
master_data.dtypes

County Name                                     object
State                                           object
COVID Cases                                      int64
COVID Deaths                                     int64
Population                                       int64
COVID Cases per 100k                           float64
COVID Deaths per 100k                          float64
Unemployment Rate (%) - Sept                    object
Poverty Rate (%)                                object
Median Household Income ($)                     object
% of adults with a college degree or higher    float64
% Male                                         float64
% Female                                       float64
% White                                        float64
% Black                                        float64
% Native American                              float64
% Asian                                        float64
% Hawaiian or Pacific Islander                 float64
% Hispanic

In [255]:
# Convert the columns to the appropriate type
master_data['Unemployment Rate (%) - Sept'] = master_data['Unemployment Rate (%) - Sept'].astype('float64')
master_data['Poverty Rate (%)'] = master_data['Poverty Rate (%)'].astype('float64')
master_data['Median Household Income ($)'] = master_data['Median Household Income ($)'].astype('int64')

In [256]:
master_data.dtypes

County Name                                     object
State                                           object
COVID Cases                                      int64
COVID Deaths                                     int64
Population                                       int64
COVID Cases per 100k                           float64
COVID Deaths per 100k                          float64
Unemployment Rate (%) - Sept                   float64
Poverty Rate (%)                               float64
Median Household Income ($)                      int64
% of adults with a college degree or higher    float64
% Male                                         float64
% Female                                       float64
% White                                        float64
% Black                                        float64
% Native American                              float64
% Asian                                        float64
% Hawaiian or Pacific Islander                 float64
% Hispanic

The Biden_or_Trump labels should be ints but because I don't have voting data for the Alaska counties, the Alaska rows show NaN which forces the column to take the type float64. There are a number of potential solutions to this problem, but I will simply set the NaN Alaska values to -1 and convert the column to int64. Then I will remove the Alaska rows before fitting my binary classification model.

In [257]:
# Recode Alaska values as -1 and cast column to int64
master_data['Biden_or_Trump'] = master_data['Biden_or_Trump'].apply(lambda x: 
                                                                    -1 if pd.isnull(x) else x).astype('int64')

In [259]:
master_data.dtypes

County Name                                     object
State                                           object
COVID Cases                                      int64
COVID Deaths                                     int64
Population                                       int64
COVID Cases per 100k                           float64
COVID Deaths per 100k                          float64
Unemployment Rate (%) - Sept                   float64
Poverty Rate (%)                               float64
Median Household Income ($)                      int64
% of adults with a college degree or higher    float64
% Male                                         float64
% Female                                       float64
% White                                        float64
% Black                                        float64
% Native American                              float64
% Asian                                        float64
% Hawaiian or Pacific Islander                 float64
% Hispanic

In [260]:
master_data.head()

,County Name,State,COVID Cases,COVID Deaths,Population,COVID Cases per 100k,COVID Deaths per 100k,Unemployment Rate (%) - Sept,Poverty Rate (%),Median Household Income ($),...,% Asian,% Hawaiian or Pacific Islander,% Hispanic or Latino,Population Density (per sq. mi),Median Age,% Veteran,Population Change (%),Trump %,Biden %,Biden_or_Trump
countyFIPS,,,,,,,,,,,,,,,,,,,,,
01001,Autauga County,AL,2173,30,55869,3889.46,53.70,5.2,13.8,59338,...,1.16,0.07,2.99,94,37.8,12.28,2.33,0.715077,0.269547,0
01003,Baldwin County,AL,6966,71,223234,3120.49,31.81,5.5,9.8,57588,...,1.05,0.06,4.72,141,42.8,11.94,22.48,0.762357,0.223452,0
01005,Barbour County,AL,1061,9,24686,4297.98,36.46,8.6,30.9,34382,...,0.47,0.13,4.52,28,39.9,7.67,-10.09,0.536005,0.456345,0
01007,Bibb County,AL,878,15,22394,3920.69,66.98,6.6,21.8,46064,...,0.21,0.03,2.78,36,39.9,7.15,-2.27,0.784289,0.207041,0
01009,Blount County,AL,2095,25,57826,3622.94,43.23,4.0,13.2,50412,...,0.28,0.04,9.65,90,40.8,8.53,0.88,0.895699,0.095670,0


In [261]:
# Save master_data
master_data.to_csv('Data/master_data.csv')